In [1]:
import Book
import time
import numpy as np

buffer  = Book.SlidingBook(num_pages=5, page_size=5)

In [2]:
buffer.add_packet(np.array([0, b'message0', b'MAC', time.time()]))
buffer.pages[0].packets

array([[b'0', b'message0', b'MAC', b'1725201907.101372'],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]], dtype=object)

In [3]:
buffer.pages[0].is_full()
print(buffer.get_page_index())


buffer.add_packet(np.array([26, b'message1', b'MAC', time.time()]))

[0]


array([[b'0', b'message0', b'MAC', b'1725201907.101372'],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]], dtype=object)

In [4]:
buffer.get_page_index()

array([], dtype=float64)